This series of notebooks is intended to develop and/or explore various elements of work needed to create a Network Threat Detection System ("NTDS").  The approach being used is traffic capture, anomaly detection, development of threat models, online refinement of models, development of an alert manager UI, and alert broadcasting.  We expect to implement a product architecture that looks something like 
this:

![ntds_arch.png](/files/ntds_arch.png)

The development of the elements will be distributed across several notebooks like this:</p>

* Primus -- Explores and implements Network Traffic Anomaly Detection. 
* Secundus -- Implements a streaming solution for aggregating data from chosen network switches/sources using Kafka and SparkStreaming
* Tertius -- Uses pyspark to implement a threat identification, threat-model, model refinement loop for creating and improving threat models over time.
* Quartus -- Develops and refines the Alert Manager GUI 
* Quintus -- Develops and refines the Alert Broadcast module

Implementing all of the pieces of this project will take some time. FOr the purpose of the X-Challenge project, only the "Primus" notebook will be completed.

<i>Note: There is "starter notebook" in the Data Bricks Samples Menagerie about "Threat Detection using DNS Data".  The approach there focuses on recognizing manglings of text DNS strings and checking those manglings against known exploits like phishing scams. The approach here is completely different and is focused on the detailed properties of the data flow rather than DNS name-mangling.</i>

This notebook uses a static data set previously captured from the Universidad Del Caucau in Popayan Columbia. This dataset was captured from network switches at the university using an open source tool (https://github.com/jsrojas/FlowLabeler) in April and May of 2017.  The dataset was freely posted on the "Kaggle" data science website (see https://www.kaggle.com/datasets/jsrojas/ip-network-traffic-flows-labeled-with-87-apps) and contains roughly 3.6 million rows of data.  Each row has 87 statistical attributes ("micro-features") characterizing the traffic and is labeled by application family ("Google", "Facebook", "Apple", "Windows Update", "Skype", and so forth).  For convenience the data has been preloaded into a pandas dataframe and saved dill-contained pandas dataframe.  <i>That</i> file has been uploaded to DataBricks as <i>Dataset-Unicauca-Version2-87Atts.dill.gz</i> and will be used in lieu of a streaming data source for the purpose of developing the ideas for Anomaly Detection in this notebook.

In [1]:
import os, sys, time, math, gzip
import pandas as pd                                                                                 
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from math import pi
import pyspark
from pyspark import SparkFiles
from pyspark import SparkContext
from pyspark.sql import Row
from pyspark.sql import SQLContext
#========================================
!python -m pip install --upgrade pip
#========================================
try:
    import json
except:
    !pip install --upgrade json
    import json
#========================================
try:
    import dill
except:
    !pip install --upgrade dill
    import dill
#========================================
try:
    import sklearn
except:
    !pip install --upgrade sklearn
finally:
    from sklearn.ensemble import RandomForestRegressor
    from sklearn.ensemble import RandomForestClassifier
#========================================
try:
    import fastparquet, pyarrow
except:
    !pip install --upgrade fastparquet
    !pip install --upgrade pyarrow
    import fastparquet, pyarrow
#========================================
try:
    import requests
except:
    !pip install --upgrade requests
    import requests
#========================================
try:
    import urllib.request
except:
    !pip install --upgrade urllib3
    import urllib.request
#========================================
try:
    import gdown
except:
    !pip install --upgrade gdown
    import gdown
#========================================
try:
    from tqdm import faux_tqdm
except:
    !pip install --upgrade tqdm
    from tqdm import tqdm    
#========================================
try:
    import geopandas as gpd
except:
    !pip install --upgrade geopandas
    import geopandas as gpd
#==========================================
try:
    from scipy.stats import mode
except:
    !pip install --upgrade scipy
    from scipy.stats import mode
#===========================================
from bptools.utils.storage import places

In [2]:
## Lots of functions...
def public_address(ipaddr) -> bool:
    """
    Small function for identifying whether an IP address is 'public'
    (as opposed to 'private')

    """
    private_starts = [
        "10.", "192.168", "172.16.", "172.17.", "172.18.", "172.19.",
        "172.20.", "172.21.", "172.22.", "172.23.", "172.24.", "172.25.",
        "172.26.", "172.27.", "172.28.", "172.29.", "172.30.", "172.31.",
    ]
    for test in private_starts:
        if (ipaddr.startswith(test)):
            return False
    return True

def loader(data_name:str = "Dataset-Unicauca-Version2-87Atts.parquet.gzip", 
           verbose:bool = False,
          ) -> pd.DataFrame:
    """
    Function which fetches the traffic data...

    """
    if (verbose):
        bt = time.time()
    data_path = os.path.join(places("datasets"), data_name)
    if (not os.path.exists(data_path)):
        raise RuntimeError(f"could not find {data_path}")
    else:
        if (verbose):
            print(f"found {data_path}, loading...")
        df = pd.read_parquet(data_path)
    #
    # okay so we have the data, let's read it..
    #
    if (verbose):
        et = time.time()
        print(f"read dataframe with shape {df.shape} in {round(et - bt, 3)} secs")
    return df

def find_location(ipaddr:str) -> {}:
    """
    given an IP address, search for information about the location
    associated with it.

    @TODO please improve this documentation

    """
    #
    # super trivial case: ip-api.com
    #
    search = f"http://ip-api.com/json/{ipaddr}"
    request = urllib.request.Request(search)
    response = urllib.request.urlopen(request).read()
    location = json.loads(response.decode("utf-8"))
    lkeys = list(location.keys())
    #
    # the information in from ip-api.com *can* be quite good
    # but if the ORG is the same as the ISP then it's probably
    # only giving us the ISP's physical location information...
    #
    location["trust"] = False
    if ("isp" in lkeys) and ("org" in lkeys):
        if (location["isp"] != location["org"]):
            location["trust"] = True
    return location

def find_country(ipaddr:str) -> str:
    # country determinations are > 99.8% reliable
    location = find_location(ipaddr)
    return location["countryCode"]

def find_region(ipaddr:str) -> str:
    location = find_location(ipaddr)
    return location["region"]

def find_city(ipaddr:str) -> str:
    location = find_location(ipaddr)
    return location["city"]

def find_latlon(ipaddr:str) -> (float, float):
    location = find_location(ipaddr)
    return (location["lat"], location["lon"])

class CacheError(Exception):
    pass

class Cache:
    """
    A class for maintaining a caching memory of IP addresses and
    a modest amount of metadata.
    
    @TODO: will want to allow for locking so as to provide for 
           some protection during multithreading...

    """
    def __init__(self,
                 cache_name:str = "cache.dill",
                 overwrite:bool = False,
                 verbose:bool = False,
                 debug:bool = False,
                 threshold:int = 100,
                 throttled:bool = True,
                 interval:float = 3.0, 
                ):
        """
        """
        self.overwrite = overwrite
        self.verbose = verbose
        self.debug = debug
        self.threshold = threshold
        self.record_count = 0
        self.throttled = throttled
        self.interval = interval
        self.data_path = os.path.join(places("datasets"), cache_name)

        if (os.path.exists(self.data_path)) and (not self.overwrite):
            self.load()
            
        if (not os.path.exists(self.data_path)) or (self.overwrite):
            if (os.path.exists(self.data_path)):
                os.remove(self.data_path)
            self.data = {}

    def search(self, ipaddr:str) -> {}:
        """

        @TODO: consider the how the keys() is used...

        """
        if (not ipaddr in self.data.keys()):
            if (self.verbose) or (self.debug):
                print(f"we haven't seen {ipaddr} before, looking for it...")
            #
            # well we haven't seen this one before so look it up...
            #
            expected_wait = time.time() + self.interval
            self.data[ipaddr] = find_location(ipaddr)
            #
            # we're using the FREE version of the API ('throttled' == True)
            # in principle we're only allowed to make 45 reqs/min -- so no
            # faster than 1.33s / query...
            #
            keep_waiting = time.time() - expected_wait
            if ((keep_waiting < 0) and (self.throttled)):
                time.sleep(-keep_waiting)
            #
            # ...initialize a reference counter...
            #
            self.data[ipaddr]["references"] = 0
            self.record_count += 1
        elif ((ipaddr in self.data.keys()) and (self.data[ipaddr]["status"] == "success")):
            if (self.debug):
                print(f"we HAVE seen {ipaddr} before and read it successfully...")
            #
            # just increment the reference counter if we already
            # have a trusted valid read...
            #
            self.data[ipaddr]["references"] += 1
        elif ((ipaddr in self.data.keys()) and (self.data[ipaddr]["status"] != "success")):
            if (self.debug):
                print(f"we HAVE seen {ipaddr} before and but FAILED to read it successfully...")
            #
            # we have seen it before but we have NOT read it successfully, so let's try
            # to read it again...
            #
            test_data = find_location(ipaddr)
            if (test_data["status"] == "success"):
                if (self.debug):
                    print(f"we now have valid data for {ipaddr}, updating")
                #
                # when we get valid data overwrite what was there before...
                #
                for key in test_data.keys():
                    self.data[ipaddr][key] = test_data[key]
            self.data[ipaddr]["references"] += 1
        #
        # by now there definitely is a record in hand...
        #
        target = self.data[ipaddr]
        #
        # regardless, update the reference counter...
        #
        if (self.record_count == self.threshold):
            print(f"have read {self.record_count} new addresses, updating cache")
            #
            # save cache to disk every 'self.threshold' records...
            #
            self.record_count = 0
            self.dump()
            self.load()
        #
        # so return it...
        #
        if (self.verbose) or (self.debug):
            if (target['status'] == 'success'):
                message = (f"ipaddr {ipaddr} is from " +
                           f"{target['city']}, " +
                           f"{target['region']}, " +
                           f"{target['countryCode']}")
                print(message)
            elif (target['message'] == 'private range'):
                print("this is a private (internal) address")
        return target

    def load(self):
        """
        @TODO: please improve this documentation

        """
        self.data = dill.load(open(self.data_path, "rb"))
        return None

    def dump(self):
        """
        @TODO: please improve this documentation

        """
        dill.dump(self.data, open(self.data_path, "wb"))
        return None

    def __str__(self):
        """
        @TODO: please improve this documentation

        """
        nkeys = len(list(self.data.keys()))
        strrep = ("<Cache " +
                  f"\n\tdata_path = {self.data_path}," + 
                  f"\n\toverwrite = {self.overwrite}," + 
                  f"\n\tverbose = {self.verbose}," + 
                  f"\n\tdebug = {self.debug}," + 
                  f"\n\tthreshold = {self.threshold}," + 
                  f"\n\tdata = <dict with {nkeys} unique addresses>" +
                  f"\n\t\>")
        return strrep

    def __repr__(self):
        """
        @TODO: please improve this documentation

        """
        return __str__()

def traffic_proto_ranking(traffic_proto):    
    #
    # define them...
    #
    utp = np.unique(traffic_proto)
    #
    # initialize a dictionary...
    #
    utp_freqs = {}
    for key in utp:
        if (key not in utp_freqs.keys()):
            utp_freqs[key] = 0
    #
    # count them...
    #
    for tp in traffic_proto:
        utp_freqs[tp] += 1
    #
    # rank them...
    #
    ranking = []
    for key in utp_freqs.keys():
        ranking.append((utp_freqs[key], key))
    sranking = sorted(ranking, reverse = True)
    total = np.sum([s for (s,p) in sranking])
    nsranking = [(100*s/total,p) for (s,p) in sranking]
    return nsranking

def display_metrics(population:{}, min_count:int = 1, num_bins = 20, show_histograms:bool = False, show_table:bool = True):
    """
    """
    if (show_table):
        print("ProtocolName        median(F)    median(D)     median(V)     median(A)     median(R)     median(P)")
        print("----------------- ------------ ------------  ------------  ------------  ------------  ------------")
    
    for proto in population.keys():
        if ((show_table) and (population[proto]["metrics"]["flowrate"]["count"] >= min_count)):
            print("%17s %12d %12d %12d %12d %12d %12d" % 
                  ((proto+"                 ")[0:17],
                   population[proto]["metrics"]["flowrate"]["median"],
                   population[proto]["metrics"]["duration"]["median"],
                   population[proto]["metrics"]["volume"]["median"],
                   population[proto]["metrics"]["avpktsz"]["median"],
                   population[proto]["metrics"]["dnuprat"]["median"],
                   population[proto]["metrics"]["pktlenvar"]["median"],
                  )
            )
        #
        # we're going to make 1 figure for each quantity for which we have at least
        # 1000 observations...
        #
        main_title = 24
        pane_title = 18
        pane_text = 16
        if (population[proto]["metrics"]["flowrate"]["count"] >= min_count):
            #
            # create the overall image and add the title...
            #
            fig, ax = plt.subplots(3, 3, figsize=(30, 30))
            plt.suptitle(f"Key Histograms for {proto}", fontsize = main_title)
            #
            # 1st col, 1st row...
            #
            ax[0][0].set_title("Flow Rate (bytes/sec)", fontsize = pane_title)
            ax[0][0].set_xlabel("Bin", fontsize = pane_text)
            ax[0][0].set_ylabel("log10(Counts)", fontsize = pane_text)
            ax[0][0].set_yscale('log')
            ax[0][0].hist(population[proto]["metrics"]["flowrate"]["data"], color = "blue",  bins = num_bins)
            #
            # 1st col, 2nd row...
            #
            ax[0][1].set_title("Duration (secs)", fontsize = pane_title)
            ax[0][1].set_yscale('log')
            ax[0][1].hist(population[proto]["metrics"]["duration"]["data"], color = "red", bins = num_bins)
            ax[0][1].set_xlabel("Bin", fontsize = pane_text)
            ax[0][1].set_ylabel("log10(Counts)", fontsize = pane_text)
            #
            # 1st col, 3rd row...
            #
            ax[0][2].set_title("Volume (bytes)", fontsize = pane_title)
            ax[0][2].set_yscale('log')
            ax[0][2].hist(population[proto]["metrics"]["volume"]["data"], color = "green", bins = num_bins)
            ax[0][2].set_xlabel("Bin", fontsize = pane_text)
            ax[0][2].set_ylabel("log10(Counts)", fontsize = pane_text)
            #
            # 2nd col, 1st row...
            #
            ax[1][0].set_title("Average Packet Size (bytes)", fontsize = pane_title)
            ax[1][0].set_xlabel("Bin", fontsize = pane_text)
            ax[1][0].set_ylabel("log10(Counts)", fontsize = pane_text)
            ax[1][0].set_yscale('log')
            ax[1][0].hist(population[proto]["metrics"]["avpktsz"]["data"], color = "magenta",  bins = num_bins)
            #
            # 2nd col, 2nd row...
            #
            ax[1][1].set_title("Down Up Ratio", fontsize = pane_title)
            ax[1][1].set_yscale('log')
            ax[1][1].hist(population[proto]["metrics"]["dnuprat"]["data"], color = "cyan", bins = num_bins)
            ax[1][1].set_xlabel("Bin", fontsize = pane_text)
            ax[1][1].set_ylabel("log10(Counts)", fontsize = pane_text)
            #
            # 2nd col, 3rd row...
            #
            ax[1][2].set_title("Packet Length Variance", fontsize = pane_title)
            ax[1][2].set_yscale('log')
            ax[1][2].hist(population[proto]["metrics"]["pktlenvar"]["data"], color = "brown", bins = num_bins)
            ax[1][2].set_xlabel("Bin", fontsize = pane_text)
            ax[1][2].set_ylabel("log10(Counts)", fontsize = pane_text)
            #
            # 3rd col, 1st row...
            #
            ax[2][0].set_title("Forward Packet Length Mean (bytes)", fontsize = pane_title)
            ax[2][0].set_xlabel("Bin", fontsize = pane_text)
            ax[2][0].set_ylabel("log10(Counts)", fontsize = pane_text)
            ax[2][0].set_yscale('log')
            ax[2][0].hist(population[proto]["metrics"]["fwdpktlen"]["data"], color = "orange",  bins = num_bins)
            #
            # 3rd col, 2nd row...
            #
            ax[2][1].set_title("Backward Packet Length Mean (bytes)", fontsize = pane_title)
            ax[2][1].set_yscale('log')
            ax[2][1].hist(population[proto]["metrics"]["bwdpktlen"]["data"], color = "lightgreen", bins = num_bins)
            ax[2][1].set_xlabel("Bin", fontsize = pane_text)
            ax[2][1].set_ylabel("log10(Counts)", fontsize = pane_text)
            #
            # 3rd col, 3rd row...
            #
            ax[2][2].set_title("Active Mean", fontsize = pane_title)
            ax[2][2].set_yscale('log')
            ax[2][2].hist(population[proto]["metrics"]["actmean"]["data"], color = "indigo", bins = num_bins)
            ax[2][2].set_xlabel("Bin", fontsize = pane_text)
            ax[2][2].set_ylabel("log10(Counts)", fontsize = pane_text)
            #
            # save/display or whatever...
            #
            plt.savefig(os.path.join(places("images"), f"{proto}_histogram.png"))
            if (show_histograms):
                plt.show()
            else:
                plt.close()
    return None

def display_world(slatlon, dlatlon, traffic_color, dur, bps, lons, lats, show_world = False):
    """
    """
    all_traffic_image = os.path.join(places("images"), "all_traffic.png")
    if (show_world == False) and (os.path.exists(all_traffic_image)):
        print("...found existing {all_traffic_image} so skipping rest of display_world(...)")
        return None
    
    print("...creating graphic of world traffic")
    fig, ax = plt.subplots(2, 1, figsize = (24, 24))
    worldmap = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))
    worldmap.plot(color = "lightgrey", ax = ax[0])
    #
    # the inbound traffic map...
    #
    main_title = 24
    pane_title = 18
    pane_text = 16
    plt.suptitle("Network Traffic", fontsize = main_title)
    ax[0].set_title("Inbound", fontsize = pane_title)
    ax[0].set_xlim((-180.0, 180.0))
    ax[0].set_xlabel("Longitude", fontsize = pane_text)
    ax[0].set_ylim((-90.0, 90.0))
    ax[0].set_ylabel("Latitude", fontsize = pane_text)
    ax[0].scatter(lons, lats, marker = ",", s = 1, alpha = 0.10, color = "black")
    #
    # the traffic direction vectors...
    #
    print("...overlaying inbound traffic vectors")
    for i in tqdm(range(len(bps))):
        if (traffic_color[i] == "green"):
            ax[0].plot([slatlon[i][0], dlatlon[i][0]], [slatlon[i][1], dlatlon[i][1]],
                       color = traffic_color[i], lw = dur[i], alpha = bps[i])
    #
    # the outbound traffic map...
    #
    worldmap = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))
    worldmap.plot(color = "lightgrey", ax = ax[1])
    #
    # the various endpoints...
    #
    ax[1].set_title("Outbound", size = pane_title)
    ax[1].set_xlim((-180.0, 180.0))
    ax[1].set_xlabel("Longitude", size = pane_text)
    ax[1].set_ylim((-90.0, 90.0))
    ax[1].set_ylabel("Latitude", size = pane_text)
    ax[1].scatter(lons, lats, marker = ",", s = 1, alpha = 0.10, color = "black")
    #
    # the traffic direction vectors...
    #
    print("...overlaying outbound traffic vectors")
    for i in tqdm(range(len(bps))):
        if (traffic_color[i] == "red"):
            ax[1].plot([slatlon[i][0], dlatlon[i][0]], [slatlon[i][1], dlatlon[i][1]],
                       color = traffic_color[i], lw = dur[i], alpha = bps[i])
    print("...saving completed figure")
    plt.savefig(all_traffic_image)
    if (show_world):
        plt.show()
    else:
        plt.close()
    return None

def populate_cache(world_graphics:bool = True, 
                   metrics_graphics:bool = True,
                   base_latlon:(float,float) = (-76.60532, 2.44091),
                  ):
    """
    """
    print("load addresses cache...")
    cache = Cache()
    keys = sorted(list(cache.data.keys()))
    lats = [cache.data[key]["lat"] for key in keys]
    lons = [cache.data[key]["lon"] for key in keys]
    print(f"...loaded {len(keys)} addresses")
    print(f"len(lons) = {len(lons)}")
    print(f"len(lats) = {len(lats)}")  
    #
    # load the traffic data...
    #
    print("load traffic data...")
    traffic = loader(verbose = True)
    print(f"...loaded {len(traffic)} records")
    #
    # sort the data to find traffic...
    #
    slatlon = []
    dlatlon = []
    bps = []
    dur = []
    traffic_color = []
    traffic_proto = []
    num_records = len(traffic)
    for i in tqdm(range(num_records)):
        sip = traffic['Source.IP'].values[i]
        dip = traffic['Destination.IP'].values[i]
        keeper = False
        if (public_address(sip)) and (not public_address(dip)):
            # this is INBOUND traffic...
            keeper = True
            cache.search(sip)
            traffic_color.append("green")
            slatlon.append((cache.data[sip]["lon"], cache.data[sip]["lat"]))
            dlatlon.append(base_latlon)
        elif (not public_address(sip)) and (public_address(dip)):
            # this is OUTBOUND traffic...
            keeper = True
            cache.search(dip)
            traffic_color.append("red")
            slatlon.append(base_latlon)
            dlatlon.append((cache.data[dip]["lon"], cache.data[dip]["lat"]))
        #
        # now if this *is* a keeper let's append various things...
        #
        if (keeper):
            bps.append(traffic['Flow.Bytes.s'].values[i])
            dur.append(traffic['Flow.Duration'].values[i])
            traffic_proto.append(traffic['ProtocolName'].values[i])
    #
    # status update...
    #
    print(f"...recorded data for {len(slatlon)} transactions")
    #
    # transparency of line will be based on traffic volume...
    #
    bps = [math.log10(b+10) for b in bps]
    bps_max = np.max(bps)
    bps = [b/bps_max for b in bps]
    print(f"normalized bps = ({np.min(bps)}, {np.median(bps)}, {np.max(bps)})")
    #
    # width of line will be based on duration of transaction...
    #
    dur = [math.log10(d+10) for d in dur]
    dur_max = np.max(dur)
    dur = [d/dur_max for d in dur]
    print(f"normalized dur = ({np.min(dur)}, {np.median(dur)}, {np.max(dur)})")
    print("")
    #
    # perhaps do pretty graphics...
    #
    print("...creating global traffic map")
    if (world_graphics):
        display_world(slatlon, dlatlon, traffic_color, dur, bps, lons, lats, show_world = False)
    #
    # get the traffic protocol ranking...
    #
    print("...computing traffic protocol rankings")
    ranking = traffic_proto_ranking(traffic_proto)
    #
    # nearest integer function because (incredibly) there is none in numpy...
    #
    def nint(x:float):
        return int(x + 0.5)
    pct_total = 0.0
    population = {}
    for i in range(0, len(ranking)):
        (percentage, proto) = ranking[i]
        chance = nint(100.0 / percentage)
        pct_total += percentage
        print("pct_total: %9.5f%%, 1/likelihood: %6d, cumulative_pct: %9.5f%%, count: %6d, proto: '%s'" %
              (percentage, chance, pct_total, int((percentage/100)*len(dur)), proto))
        population[proto] = {
            "rank":i,
            "pct_total":pct_total,
            "inv_likelihood": chance,
            "cumulative_pct":pct_total,
            "metrics":{
                "path":{
                    "src":[],
                    "dst":[],
                },
                "flowrate":{
                    "data":[],
                    "count":0,
                    "mode":-1,
                    "median":-1,
                    "mean":-1,
                    "std":-1,
                    "threshold":-1,
                },
                "duration":{
                    "data":[],
                    "count":0,
                    "mode":-1,
                    "median":-1,
                    "mean":-1,
                    "std":-1,
                    "threshold":-1,
                },
                "volume":{
                    "data":[],
                    "count":0,
                    "mode":-1,
                    "median":-1,
                    "mean":-1,
                    "std":-1,
                    "threshold":-1,
                },
                "avpktsz":{ #"Average.Packet.Size"
                    "data":[],
                    "count":0,
                    "mode":-1,
                    "median":-1,
                    "mean":-1,
                    "std":-1,
                    "threshold":-1,
                },
                "dnuprat":{#"Down.Up.Ratio"
                    "data":[],
                    "count":0,
                    "mode":-1,
                    "median":-1,
                    "mean":-1,
                    "std":-1,
                    "threshold":-1,
                },
                "pktlenvar":{#"Packet.Length.Variance"
                    "data":[],
                    "count":0,
                    "mode":-1,
                    "median":-1,
                    "mean":-1,
                    "std":-1,
                    "threshold":-1,
                },
                "fwdpktlen":{#"Forward.Packet.Length.Mean"
                    "data":[],
                    "count":0,
                    "mode":-1,
                    "median":-1,
                    "mean":-1,
                    "std":-1,
                    "threshold":-1,
                },
                "bwdpktlen":{#"Backward.Packet.Length.Mean"
                    "data":[],
                    "count":0,
                    "mode":-1,
                    "median":-1,
                    "mean":-1,
                    "std":-1,
                    "threshold":-1,
                },
                "actmean":{#"Active.Mean"
                    "data":[],
                    "count":0,
                    "mode":-1,
                    "median":-1,
                    "mean":-1,
                    "std":-1,
                    "threshold":-1,
                },
            },
        }
    #
    # now compute population metrics by walking through the traffic data...
    #
    print("\n...gathering population metrics data for flowrate and duration as function of protocol")
    for i in range(0, len(traffic)):
        proto     = traffic['ProtocolName'].values[i]
        flowrate  = traffic['Flow.Bytes.s'].values[i]
        duration  = traffic['Flow.Duration'].values[i]
        avpktsz   = traffic["Average.Packet.Size"].values[i]
        dnuprat   = traffic["Down.Up.Ratio"].values[i]
        pktlenvar = traffic["Packet.Length.Variance"].values[i]
        volume    = flowrate * duration
        fwdpktlen = traffic["Fwd.Packet.Length.Mean"].values[i]
        bwdpktlen = traffic["Bwd.Packet.Length.Mean"].values[i]
        actmean   = traffic["Active.Mean"].values[i]
        sip       = traffic['Source.IP'].values[i]
        dip       = traffic['Destination.IP'].values[i]
        if ((public_address(sip) and not public_address(dip)) or
            ((not public_address(sip)) and public_address(dip))):
            #
            # then it's an off-site transaction... so we can work with it...
            #
            population[proto]["metrics"]["flowrate"]["data"].append(flowrate)
            population[proto]["metrics"]["duration"]["data"].append(duration)
            population[proto]["metrics"]["volume"]["data"].append(volume)
            population[proto]["metrics"]["avpktsz"]["data"].append(avpktsz)
            population[proto]["metrics"]["dnuprat"]["data"].append(dnuprat)
            population[proto]["metrics"]["pktlenvar"]["data"].append(pktlenvar)
            population[proto]["metrics"]["fwdpktlen"]["data"].append(fwdpktlen)
            population[proto]["metrics"]["bwdpktlen"]["data"].append(bwdpktlen)
            population[proto]["metrics"]["actmean"]["data"].append(actmean)
            if (public_address(sip) and (not public_address(dip))):
                population[proto]["metrics"]["path"]["src"].append((cache.data[sip]["lon"], cache.data[sip]["lat"]))
                population[proto]["metrics"]["path"]["dst"].append(base_latlon)
            elif ((not public_address(sip)) and public_address(dip)):
                population[proto]["metrics"]["path"]["src"].append(base_latlon)
                population[proto]["metrics"]["path"]["dst"].append((cache.data[dip]["lon"], cache.data[dip]["lat"]))
    #
    # compute key statistics regarding the metrics...
    #
    print("...computing population metrics")
    for proto in population.keys():
        #
        # flowrate...
        #
        population[proto]["metrics"]["flowrate"]["count"]  = len(population[proto]["metrics"]["flowrate"]["data"])
        population[proto]["metrics"]["flowrate"]["threshold"] = 1.0 / float(population[proto]["metrics"]["flowrate"]["count"])
        population[proto]["metrics"]["flowrate"]["mode"]   = mode(population[proto]["metrics"]["flowrate"]["data"]).mode[0]
        population[proto]["metrics"]["flowrate"]["median"] = np.median(population[proto]["metrics"]["flowrate"]["data"])
        population[proto]["metrics"]["flowrate"]["mean"]   = np.mean(population[proto]["metrics"]["flowrate"]["data"])
        population[proto]["metrics"]["flowrate"]["std"]    = np.std(population[proto]["metrics"]["flowrate"]["data"])
        #
        # duration...
        #
        population[proto]["metrics"]["duration"]["count"]  = len(population[proto]["metrics"]["duration"]["data"])
        population[proto]["metrics"]["duration"]["threshold"] = 1.0 / float(population[proto]["metrics"]["duration"]["count"])
        population[proto]["metrics"]["duration"]["mode"]   = mode(population[proto]["metrics"]["duration"]["data"]).mode[0]
        population[proto]["metrics"]["duration"]["median"] = np.median(population[proto]["metrics"]["duration"]["data"])
        population[proto]["metrics"]["duration"]["mean"]   = np.mean(population[proto]["metrics"]["duration"]["data"])
        population[proto]["metrics"]["duration"]["std"]    = np.std(population[proto]["metrics"]["duration"]["data"])
        #
        # volume...
        #
        population[proto]["metrics"]["volume"]["count"]  = len(population[proto]["metrics"]["volume"]["data"])
        population[proto]["metrics"]["volume"]["threshold"] = 1.0 / float(population[proto]["metrics"]["volume"]["count"])
        population[proto]["metrics"]["volume"]["mode"]   = mode(population[proto]["metrics"]["volume"]["data"]).mode[0]
        population[proto]["metrics"]["volume"]["median"] = np.median(population[proto]["metrics"]["volume"]["data"])
        population[proto]["metrics"]["volume"]["mean"]   = np.mean(population[proto]["metrics"]["volume"]["data"])
        population[proto]["metrics"]["volume"]["std"]    = np.std(population[proto]["metrics"]["volume"]["data"])
        #
        # avpktsz...
        #
        population[proto]["metrics"]["avpktsz"]["count"]  = len(population[proto]["metrics"]["avpktsz"]["data"])
        population[proto]["metrics"]["avpktsz"]["threshold"] = 1.0 / float(population[proto]["metrics"]["avpktsz"]["count"])
        population[proto]["metrics"]["avpktsz"]["mode"]   = mode(population[proto]["metrics"]["avpktsz"]["data"]).mode[0]
        population[proto]["metrics"]["avpktsz"]["median"] = np.median(population[proto]["metrics"]["avpktsz"]["data"])
        population[proto]["metrics"]["avpktsz"]["mean"]   = np.mean(population[proto]["metrics"]["avpktsz"]["data"])
        population[proto]["metrics"]["avpktsz"]["std"]    = np.std(population[proto]["metrics"]["avpktsz"]["data"])
        #
        # dnuprat...
        #
        population[proto]["metrics"]["dnuprat"]["count"]  = len(population[proto]["metrics"]["dnuprat"]["data"])
        population[proto]["metrics"]["dnuprat"]["threshold"] = 1.0 / float(population[proto]["metrics"]["dnuprat"]["count"])
        population[proto]["metrics"]["dnuprat"]["mode"]   = mode(population[proto]["metrics"]["dnuprat"]["data"]).mode[0]
        population[proto]["metrics"]["dnuprat"]["median"] = np.median(population[proto]["metrics"]["dnuprat"]["data"])
        population[proto]["metrics"]["dnuprat"]["mean"]   = np.mean(population[proto]["metrics"]["dnuprat"]["data"])
        population[proto]["metrics"]["dnuprat"]["std"]    = np.std(population[proto]["metrics"]["dnuprat"]["data"])
        #
        # fwdpktlen...
        #
        population[proto]["metrics"]["fwdpktlen"]["count"]  = len(population[proto]["metrics"]["fwdpktlen"]["data"])
        population[proto]["metrics"]["fwdpktlen"]["threshold"] = 1.0 / float(population[proto]["metrics"]["fwdpktlen"]["count"])
        population[proto]["metrics"]["fwdpktlen"]["mode"]   = mode(population[proto]["metrics"]["fwdpktlen"]["data"]).mode[0]
        population[proto]["metrics"]["fwdpktlen"]["median"] = np.median(population[proto]["metrics"]["fwdpktlen"]["data"])
        population[proto]["metrics"]["fwdpktlen"]["mean"]   = np.mean(population[proto]["metrics"]["fwdpktlen"]["data"])
        population[proto]["metrics"]["fwdpktlen"]["std"]    = np.std(population[proto]["metrics"]["fwdpktlen"]["data"])

        #
        # avpktsz...
        #
        population[proto]["metrics"]["bwdpktlen"]["count"]  = len(population[proto]["metrics"]["bwdpktlen"]["data"])
        population[proto]["metrics"]["bwdpktlen"]["threshold"] = 1.0 / float(population[proto]["metrics"]["bwdpktlen"]["count"])
        population[proto]["metrics"]["bwdpktlen"]["mode"]   = mode(population[proto]["metrics"]["bwdpktlen"]["data"]).mode[0]
        population[proto]["metrics"]["bwdpktlen"]["median"] = np.median(population[proto]["metrics"]["bwdpktlen"]["data"])
        population[proto]["metrics"]["bwdpktlen"]["mean"]   = np.mean(population[proto]["metrics"]["bwdpktlen"]["data"])
        population[proto]["metrics"]["bwdpktlen"]["std"]    = np.std(population[proto]["metrics"]["bwdpktlen"]["data"])
        #
        # dnuprat...
        #
        population[proto]["metrics"]["actmean"]["count"]  = len(population[proto]["metrics"]["actmean"]["data"])
        population[proto]["metrics"]["actmean"]["threshold"] = 1.0 / float(population[proto]["metrics"]["actmean"]["count"])
        population[proto]["metrics"]["actmean"]["mode"]   = mode(population[proto]["metrics"]["actmean"]["data"]).mode[0]
        population[proto]["metrics"]["actmean"]["median"] = np.median(population[proto]["metrics"]["actmean"]["data"])
        population[proto]["metrics"]["actmean"]["mean"]   = np.mean(population[proto]["metrics"]["actmean"]["data"])
        population[proto]["metrics"]["actmean"]["std"]    = np.std(population[proto]["metrics"]["actmean"]["data"])
        #
        # pktlenvar...
        #
        population[proto]["metrics"]["pktlenvar"]["count"]  = len(population[proto]["metrics"]["pktlenvar"]["data"])
        population[proto]["metrics"]["pktlenvar"]["threshold"] = 1.0 / float(population[proto]["metrics"]["pktlenvar"]["count"])
        population[proto]["metrics"]["pktlenvar"]["mode"]   = mode(population[proto]["metrics"]["pktlenvar"]["data"]).mode[0]
        population[proto]["metrics"]["pktlenvar"]["median"] = np.median(population[proto]["metrics"]["pktlenvar"]["data"])
        population[proto]["metrics"]["pktlenvar"]["mean"]   = np.mean(population[proto]["metrics"]["pktlenvar"]["data"])
        population[proto]["metrics"]["pktlenvar"]["std"]    = np.std(population[proto]["metrics"]["pktlenvar"]["data"])

    #
    # now perhaps show a table summarizing some of the metrics...
    #
    if (metrics_graphics):
        display_metrics(population,
                        min_count = 100, # probably 100 for production...
                        num_bins = 10,  # still experimenting...
                        show_histograms = False,
                        show_table = True,
        )

    dill.dump(population,
              gzip.open(os.path.join(places("datasets"),
                                     "population_metrics.dill.gz"), "wb"))
    return None

def scaled_rgba_v2(this_value:float, dpower:int = 0, min_value:float = 0.0, max_value:float = 1.0):
    """
    """
    if (dpower >= 5):
        color = "red"
        alpha = (this_value - min_value)/(max_value - min_value)
    elif (dpower == 4):
        color = "orange"
        alpha = (this_value - min_value)/(max_value - min_value)
    elif (dpower == 3):
        color = "yellow"
        alpha = (this_value - min_value)/(max_value - min_value)
    elif (dpower == 2):
        color = "green"
        alpha = (this_value - min_value)/(max_value - min_value)
    elif (dpower == 1):
        color = "blue"
        alpha = (this_value - min_value)/(max_value - min_value)
    else:
        color = "indigo"
        alpha = (this_value - min_value)/(max_value - min_value)
    return (color, alpha)

def display_proto_radar(population:{},
                        min_count:int = 1000,
                        pseudo_sigmae:int = 5,
                        anomaly_strategy:str = "mean", # "mean" or "max"...
                        alpha_scaling:str = "linear",  # "sqrt", "linear", or "square"...
                        dark_background:bool = False,
                        black_facecolor:bool = False,
                        display_graphics:bool = False):
    """
    We do NOT have gaussian statistics here (things are definitely 
    NOT in a 'normal' distribution. So we're working with a sort of
    pseudo_sigma in place of standard deviation.

    We define the pseudo_sigma as follows:



    @TODO: please improve this documentation

    """
    ##
    ## the labels for the radial coordinates...
    ##
    theta_labels = [#"flowrate",
                    "duration", "volume",
                    "avpktsz",  "dnuprat",  "pktlenvar",
                    "fwdpktlen", "bwdpktlen",
                   # "actmean",
                   ]
    ##
    ## prepare the angular coordinates to be plotted...
    ##
    theta = [(2*pi*i/len(theta_labels)) for i in range(0, len(theta_labels))]
    for proto in population.keys():
        if (population[proto]["metrics"]["flowrate"]["count"] >= min_count):
            radar_plot_name = os.path.join(places("images"), f"{proto}_radar.png")
            print(f"...preparing radar plot for {proto}")
            ##
            ## prepare the radial coordinates to be plotted...
            ##
            for theta_label in theta_labels:
                num_values  = population[proto]["metrics"][theta_label]["count"]
                data        = population[proto]["metrics"][theta_label]["data"]
                data_min    = np.min(data)
                data_max    = np.max(data)
                data_mean   = population[proto]["metrics"][theta_label]["mean"]
                data_median = population[proto]["metrics"][theta_label]["median"]
                data_std    = population[proto]["metrics"][theta_label]["std"]
                data_mode   = population[proto]["metrics"][theta_label]["mode"]
                ##
                ## our definition of anomaly (as defined in terms of pseudo_sigma)
                ## may indicate that nothing IS an anomaly... that's fine...
                ##
                data_range = data_max - data_min
                data_pos   = abs(data_max - data_mode)
                data_neg   = abs(data_mode - data_min)
                if (data_pos > 0) and (data_neg > 0):
                    pseudo_sigma = np.mean([data_pos, data_neg]) / (pseudo_sigmae + 1)
                elif (data_pos > 0):
                    pseudo_sigma = data_pos / (pseudo_sigmae + 1)
                elif (data_neg > 0):
                    pseudo_sigma = data_neg / (pseudo_sigmae + 1)
                #
                # assigns an anomaly score on (0, pseudo_sigmae) to each of the various 'theta_labels' axes...
                #
                population[proto]["metrics"][theta_label]["anomaly_score"] = [abs(
                        d-data_mode)/pseudo_sigma for d in data]
                #
                # if the radar plot figure doesn't already exist then create it...
                #
                if (not os.path.exists(radar_plot_name)):
                    ##
                    ## define the mean anomaly score for setting the color of the curves...
                    ##
                    anomaly_score = []
                    for i in range(0, len(population[proto]["metrics"][theta_labels[0]]["data"])):
                        if (anomaly_strategy == "mean"):
                            anomaly_score.append(np.mean(
                                [population[proto]["metrics"][theta_label]["anomaly_score"][i] for theta_label in theta_labels]))
                        elif (anomaly_strategy == "max"):
                            anomaly_score.append(np.max(
                                [population[proto]["metrics"][theta_label]["anomaly_score"][i] for theta_label in theta_labels]))
                    ##
                    ## prepare the radial coordinates to be plotted...
                    ##
                    radius = []
                    for i in range(0, len(population[proto]["metrics"][theta_labels[0]]["data"])):
                        radius.append(
                            [population[proto]["metrics"][theta_label]["anomaly_score"][i] for theta_label in theta_labels])
                    ##
                    ## create the figure...
                    ##
                    fig = plt.figure(figsize=(20,16))
                    ax = fig.add_subplot(111, polar=True)
                    ax.grid(True, lw=0.25)
                    ##
                    ## conditional appearance tweaks...
                    ##
                    if (dark_background):
                        plt.style.use('dark_background')
                    if (black_facecolor):
                        plt.rcParams['axes.facecolor'] = 'black'
                    plt.title(f"Radar Plots of Feature Data for {len(anomaly_score)} {proto} Transactions",
                              fontsize=24, pad=24)
                    plt.xlim((0, 2*pi))
                    ##
                    ## setting the radial scale...
                    ##
                    max_radius = math.ceil(np.max(radius))
                    ax.set_rmax(max_radius) # new version using max_radius...
                    ##
                    ## choose the radial tick-marks to be drawn
                    ##
                    ax.set_rticks([r*0.5 for r in range(0, 2 * max_radius)]) # new version using max_radius...
                    plt.xticks(theta, theta_labels, fontsize = 16)
                    ans_min = np.min(anomaly_score)
                    ans_max = np.max(anomaly_score)
                    ans_mid = ans_max/2
                    ans_med = np.median(anomaly_score)
                    ans_mod = mode(anomaly_score).mode[0]
                    ans_mea = np.mean(anomaly_score)
                    sas = sorted([(anomaly_score[i], i) for i in range(0, len(anomaly_score))], reverse = True)
                    #
                    # a different approach...
                    #
                    d1_events = int(len(sas) / 10)
                    if (d1_events > 0):
                        d1_threshold = sas[d1_events][0]
                        print(f"number of d1 events: {d1_events}, d1_threshold = {d1_threshold}")
                    else:
                        d1_threshold = 0
                    d2_events = int(len(sas) / 100)
                    if (d2_events > 0):
                        d2_threshold = sas[d2_events][0]
                        print(f"number of d2 events: {d2_events}, d2_threshold = {d2_threshold}")
                    else:
                        d2_threshold = 0
                    d3_events = int(len(sas) / 1000)
                    if (d3_events > 0):
                        d3_threshold = sas[d3_events][0]
                        print(f"number of d3 events: {d3_events}, d3_threshold = {d3_threshold}")
                    else:
                        d3_threshold = 0
                    d4_events = int(len(sas) / 10000)
                    if (d4_events > 0):
                        d4_threshold = sas[d4_events][0]
                        print(f"number of d4 events: {d4_events}, d4_threshold = {d4_threshold}")
                    else:
                        d4_threshold = 0
                    d5_events = int(len(sas) / 100000)
                    if (d5_events > 0):
                        d5_threshold = sas[d5_events][0]
                        print(f"number of d5 events: {d5_events}, d5_threshold = {d5_threshold}")
                    else:
                        d5_threshold = 0
                    #
                    # compute the colors...
                    #
                    graph_colors = []
                    graph_labels = []
                    legend_stuff = []
                    for i in range(0, len(radius)):
                        ##
                        ## define the color for this particular curve...
                        ##
                        if (d5_events > 0) and (anomaly_score[i] > d5_threshold):
                            # then it's a d5 event...
                            (color, alpha) = scaled_rgba_v2(anomaly_score[i],
                                                            dpower = 5,
                                                            min_value = ans_min,
                                                            max_value = ans_max,
                            )
                            if (color not in graph_colors):
                                graph_colors.append(color)
                                graph_labels.append("1 in 100000 event")
                        elif (d4_events > 0) and (anomaly_score[i] > d4_threshold):
                            # then it's a d4 event...        
                            (color, alpha) = scaled_rgba_v2(anomaly_score[i],
                                                            dpower = 4,
                                                            min_value = ans_min,
                                                            max_value = ans_max,
                            )
                            if (color not in graph_colors):
                                graph_colors.append(color)
                                graph_labels.append("1 in 10000 event")
                        elif (d3_events > 0) and (anomaly_score[i] > d3_threshold):
                            # then it's a d3 event...        
                            (color, alpha) = scaled_rgba_v2(anomaly_score[i],
                                                            dpower = 3,
                                                            min_value = ans_min,
                                                            max_value = ans_max,
                            )
                            if (color not in graph_colors):
                                graph_colors.append(color)
                                graph_labels.append("1 in 1000 event")
                        elif (d2_events > 0) and (anomaly_score[i] > d2_threshold):
                            # then it's a d2 event...        
                            (color, alpha) = scaled_rgba_v2(anomaly_score[i],
                                                            dpower = 2,
                                                            min_value = ans_min,
                                                            max_value = ans_max,
                            )
                            if (color not in graph_colors):
                                graph_colors.append(color)
                                graph_labels.append("1 in 100 event")
                        elif (d1_events > 0) and (anomaly_score[i] > d1_threshold):
                            # then it's a d1 event...        
                            (color, alpha) = scaled_rgba_v2(anomaly_score[i],
                                                            dpower = 1,
                                                            min_value = ans_min,
                                                            max_value = ans_max,
                            )
                            if (color not in graph_colors):
                                graph_colors.append(color)
                                graph_labels.append("1 in 10 event")
                        else:
                            # then it's just a normal event...
                            (color, alpha) = scaled_rgba_v2(anomaly_score[i],
                                                            dpower = 0,
                                                            min_value = ans_min,
                                                            max_value = ans_max,
                            )
                            if (color not in graph_colors):
                                graph_colors.append(color)
                                graph_labels.append(" common event")

                        this_theta = theta.copy()
                        this_theta.append(theta[0])
                        this_radius = radius[i].copy()
                        this_radius.append(radius[i][0])
                        if (alpha_scaling == "sqrt"):
                            plt.plot(this_theta,
                                     this_radius,
                                     lw=2*(0.20 + sqrt(alpha)),
                                     ls="-",
                                     color = color,
                                     alpha = sqrt(alpha),
                            )
                        elif (alpha_scaling == "square"):
                            plt.plot(this_theta,
                                     this_radius,
                                     lw=2*(0.20 + alpha**2),
                                     ls="-",
                                     color = color,
                                     alpha = alpha**2,
                            )
                        elif (alpha_scaling == "linear"):
                            plt.plot(this_theta,
                                     this_radius,
                                     lw=2*(0.20 + alpha),
                                     ls="-",
                                     color = color,
                                     alpha = alpha,
                            )
                        else:
                            raise RuntimeError(f"Don't grok alpha_scaling == {alpha_scaling}")
                    #
                    # create and render the legend...
                    #
                    legend_handles = []
                    for i in range(0, len(graph_colors)):
                        this_label = graph_labels[i]
                        this_color = graph_colors[i]
                        this_patch = mpatches.Patch(color = this_color, label = this_label)
                        legend_handles.append(this_patch)
                    ax.legend(loc = "lower left",
                              handles = legend_handles,
                              fontsize=14,
                    )
                    #
                    # save things...
                    #
                    plt.savefig(radar_plot_name)
                    if (display_graphics):
                        plt.show()
                    else:
                        plt.close()
                
    updated_population_path = os.path.join(places("datasets"), "updated_population_metrics.dill.gz")
    dill.dump(population, gzip.open(updated_population_path, "wb"))
    return None

def histograms_and_radar_plots(anomaly_strategy:str = "mean",
                               display_graphics:bool = False,
                               alpha_scaling:str = "square",
                               show_histograms:bool = False,
                               pseudo_sigmae:int = 5,
                               min_count:int = 100,
                               main_title:int = 24,
                               pane_title:int = 18,
                               pane_text = 16,
                               num_bins = 10,
                               population_name:str = "population_metrics.dill.gz",
                              ):
    """
    """
    print("\nrunning test_am...")
    anomaly_strategy = "mean"
    display_graphics = False
    alpha_scaling = "square"
    show_histograms = False
    pseudo_sigmae = 5
    min_count = 100 # could be 30, 100, 300, 1000, 3000, etc
    main_title = 24
    pane_title = 18
    pane_text = 16
    num_bins = 10

    ##
    ## load the already-existing data...
    ##
    population_path = os.path.join(places("datasets"), population_name)
    if (not os.path.exists(population_path)):
        raise RuntimeError(f"Yikes!  Can't find {population_path}... have you run test_cachemap.py yet?")
    
    population = dill.load(gzip.open(population_path, "rb"))
    
    colors = [
        "indigo", "darkblue", "blue",
        "darkcyan", "green", "lightgreen",
        "yellow", "orange", "red",
    ]

    print("...histograms")
    for proto in population.keys():
        #
        # we're going to make 1 figure for each quantity for which we have at least 'min_count' observations
        #
        if (population[proto]["metrics"]["flowrate"]["count"] >= min_count):
            #
            # create the overall image and add the title...
            #
            fig, ax = plt.subplots(3, 3, figsize=(30, 30))
            plt.suptitle(f"Key Histograms for {proto}", fontsize = main_title)
            #
            # 1st col, 1st row...
            #
            ax[0][0].set_title("Flow Rate (bytes/sec)", fontsize = pane_title)
            ax[0][0].set_xlabel("Bin", fontsize = pane_text)
            ax[0][0].set_ylabel("log10(Counts)", fontsize = pane_text)
            ax[0][0].set_yscale('log')
            ax[0][0].hist(population[proto]["metrics"]["flowrate"]["data"], color = colors[0],  bins = num_bins)
            #
            # 1st col, 2nd row...
            #
            ax[0][1].set_title("Duration (secs)", fontsize = pane_title)
            ax[0][1].set_yscale('log')
            ax[0][1].hist(population[proto]["metrics"]["duration"]["data"], color = colors[1], bins = num_bins)
            ax[0][1].set_xlabel("Bin", fontsize = pane_text)
            ax[0][1].set_ylabel("log10(Counts)", fontsize = pane_text)
            #
            # 1st col, 3rd row...
            #
            ax[0][2].set_title("Volume (bytes)", fontsize = pane_title)
            ax[0][2].set_yscale('log')
            ax[0][2].hist(population[proto]["metrics"]["volume"]["data"], color = colors[2], bins = num_bins)
            ax[0][2].set_xlabel("Bin", fontsize = pane_text)
            ax[0][2].set_ylabel("log10(Counts)", fontsize = pane_text)
            #
            # 2nd col, 1st row...
            #
            ax[1][0].set_title("Average Packet Size (bytes)", fontsize = pane_title)
            ax[1][0].set_xlabel("Bin", fontsize = pane_text)
            ax[1][0].set_ylabel("log10(Counts)", fontsize = pane_text)
            ax[1][0].set_yscale('log')
            ax[1][0].hist(population[proto]["metrics"]["avpktsz"]["data"], color = colors[3],  bins = num_bins)
            #
            # 2nd col, 2nd row...
            #
            ax[1][1].set_title("Down Up Ratio", fontsize = pane_title)
            ax[1][1].set_yscale('log')
            ax[1][1].hist(population[proto]["metrics"]["dnuprat"]["data"], color = colors[4], bins = num_bins)
            ax[1][1].set_xlabel("Bin", fontsize = pane_text)
            ax[1][1].set_ylabel("log10(Counts)", fontsize = pane_text)
            #
            # 2nd col, 3rd row...
            #
            ax[1][2].set_title("Packet Length Variance", fontsize = pane_title)
            ax[1][2].set_yscale('log')
            ax[1][2].hist(population[proto]["metrics"]["pktlenvar"]["data"], color = colors[5], bins = num_bins)
            ax[1][2].set_xlabel("Bin", fontsize = pane_text)
            ax[1][2].set_ylabel("log10(Counts)", fontsize = pane_text)
            #
            # 3rd col, 1st row...
            #
            ax[2][0].set_title("Forward Packet Length Mean (bytes)", fontsize = pane_title)
            ax[2][0].set_xlabel("Bin", fontsize = pane_text)
            ax[2][0].set_ylabel("log10(Counts)", fontsize = pane_text)
            ax[2][0].set_yscale('log')
            ax[2][0].hist(population[proto]["metrics"]["fwdpktlen"]["data"], color = colors[6],  bins = num_bins)
            #
            # 3rd col, 2nd row...
            #
            ax[2][1].set_title("Backward Packet Length Mean (bytes)", fontsize = pane_title)
            ax[2][1].set_yscale('log')
            ax[2][1].hist(population[proto]["metrics"]["bwdpktlen"]["data"], color = colors[7], bins = num_bins)
            ax[2][1].set_xlabel("Bin", fontsize = pane_text)
            ax[2][1].set_ylabel("log10(Counts)", fontsize = pane_text)
            #
            # 3rd col, 3rd row...
            #
            ax[2][2].set_title("Active Mean", fontsize = pane_title)
            ax[2][2].set_yscale('log')
            ax[2][2].hist(population[proto]["metrics"]["actmean"]["data"], color = colors[8], bins = num_bins)
            ax[2][2].set_xlabel("Bin", fontsize = pane_text)
            ax[2][2].set_ylabel("log10(Counts)", fontsize = pane_text)
            #
            # save/display or whatever...
            #
            plt.savefig(os.path.join(places("images"), f"{proto}_histogram.png"))
            if (show_histograms):
                plt.show()
            else:
                plt.close()

    print("...radar plots")
    display_proto_radar(population,
                        display_graphics = display_graphics,
                        min_count = min_count,
                        pseudo_sigmae = pseudo_sigmae,
                        anomaly_strategy = anomaly_strategy,
                        alpha_scaling = alpha_scaling)
    print("...done")
    return None

def create_traffic_maps_by_protocol(population_name:str = "updated_population_metrics.dill.gz",
                                    min_count:int = 1000, 
                                    anomaly_scaling:str = "linear", 
                                    show_world:bool = False,
                                    main_title:int = 24,
                                    pane_title:int = 18,
                                    pane_text:int = 16,
                                    uni_latlon:(float,float) = (-76.60532, 2.44091),
                                    labels:[str] = ["duration", "volume", "avpktsz",  "dnuprat",  
                                                    "pktlenvar", "fwdpktlen", "bwdpktlen"]):
    """
    """
    population = dill.load(gzip.open(os.path.join(places("datasets"), population_name), "rb"))    
    for proto in population.keys():
        locations = population[proto]["metrics"]["path"]
        slatlon = locations["src"]
        dlatlon = locations["dst"]
        npts = len(population[proto]['metrics']['flowrate']['data'])
        if (npts >= min_count):
            ascore = []
            tcolor = []
            for i in range(0, npts):
                asmean = np.mean([population[proto]['metrics'][label]['anomaly_score'][i] for label in labels])
                ascore.append(asmean)
            as_max = np.max(ascore)
            as_mid = as_max / 2
            as_mea = np.mean(ascore)
            as_med = np.median(ascore)
            as_mod = mode(ascore).mode[0]
            as_min = np.min(ascore)
            anomaly_score = ascore.copy()
            print(f"...ascore for {proto} contains {len(np.unique(ascore))} values with:")
            sas = sorted([(anomaly_score[i], i) for i in range(0, len(anomaly_score))], reverse = True)
            #
            # a different approach...
            #
            d1_events = int(len(sas) / 10)
            if (d1_events > 0):
                d1_threshold = sas[d1_events][0]
                print(f"number of d1 events: {d1_events}, d1_threshold = {d1_threshold}")
            else:
                d1_threshold = 0
            d2_events = int(len(sas) / 100)
            if (d2_events > 0):
                d2_threshold = sas[d2_events][0]
                print(f"number of d2 events: {d2_events}, d2_threshold = {d2_threshold}")
            else:
                d2_threshold = 0
            d3_events = int(len(sas) / 1000)
            if (d3_events > 0):
                d3_threshold = sas[d3_events][0]
                print(f"number of d3 events: {d3_events}, d3_threshold = {d3_threshold}")
            else:
                d3_threshold = 0
            d4_events = int(len(sas) / 10000)
            if (d4_events > 0):
                d4_threshold = sas[d4_events][0]
                print(f"number of d4 events: {d4_events}, d4_threshold = {d4_threshold}")
            else:
                d4_threshold = 0
            d5_events = int(len(sas) / 100000)
            if (d5_events > 0):
                d5_threshold = sas[d5_events][0]
                print(f"number of d5 events: {d5_events}, d5_threshold = {d5_threshold}")
            else:
                d5_threshold = 0
            #
            # compute the colors...
            #
            print(f"...preparing colors and labels")
            graph_colors = []
            graph_labels = []
            vector_color = []
            vector_alpha = []
            for i in range(0, npts):
                if (d5_events > 0) and (anomaly_score[i] > d5_threshold):
                    # then it's a d5 event...
                    (color, alpha) = scaled_rgba_v2(anomaly_score[i], dpower = 5, min_value = as_min, max_value = as_max)
                    vector_color.append(color)
                    vector_alpha.append(alpha)
                    if (color not in graph_colors):
                        graph_colors.append(color)
                        graph_labels.append("1 in 100000 event")
                elif (d4_events > 0) and (anomaly_score[i] > d4_threshold):
                    # then it's a d4 event...        
                    (color, alpha) = scaled_rgba_v2(anomaly_score[i], dpower = 4, min_value = as_min, max_value = as_max)
                    vector_color.append(color)
                    vector_alpha.append(alpha)
                    if (color not in graph_colors):
                        graph_colors.append(color)
                        graph_labels.append("1 in 10000 event")
                elif (d3_events > 0) and (anomaly_score[i] > d3_threshold):
                    # then it's a d3 event...        
                    (color, alpha) = scaled_rgba_v2(anomaly_score[i], dpower = 3, min_value = as_min, max_value = as_max)
                    vector_color.append(color)
                    vector_alpha.append(alpha)
                    if (color not in graph_colors):
                        graph_colors.append(color)
                        graph_labels.append("1 in 1000 event")
                elif (d2_events > 0) and (anomaly_score[i] > d2_threshold):
                    # then it's a d2 event...        
                    (color, alpha) = scaled_rgba_v2(anomaly_score[i], dpower = 2, min_value = as_min, max_value = as_max)
                    vector_color.append(color)
                    vector_alpha.append(alpha)
                    if (color not in graph_colors):
                        graph_colors.append(color)
                        graph_labels.append("1 in 100 event")
                elif (d1_events > 0) and (anomaly_score[i] > d1_threshold):
                    # then it's a d1 event...        
                    (color, alpha) = scaled_rgba_v2(anomaly_score[i], dpower = 1, min_value = as_min, max_value = as_max)
                    vector_color.append(color)
                    vector_alpha.append(alpha)
                    if (color not in graph_colors):
                        graph_colors.append(color)
                        graph_labels.append("1 in 10 event")
                else:
                    # then it's just a normal event...
                    (color, alpha) = scaled_rgba_v2(anomaly_score[i], dpower = 0, min_value = as_min, max_value = as_max)
                    vector_color.append(color)
                    vector_alpha.append(alpha)
                    if (color not in graph_colors):
                        graph_colors.append(color)
                        graph_labels.append(" common event")

            print(f"...preparing {proto} end points")
            lons = []
            lats = []
            dire = []
            for i in range(0, npts):
                if (slatlon[i] == uni_latlon):
                    lons.append(slatlon[i][0])
                    lats.append(slatlon[i][1])
                    dire.append("outbound")
            for i in range(0, npts):
                if (dlatlon[i] == uni_latlon):
                    lons.append(dlatlon[i][0])
                    lats.append(dlatlon[i][1])
                    dire.append("inbound")
            #
            # okay show time!
            #
            fig, ax = plt.subplots(2, 1, figsize = (32, 36))
            worldmap = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))
            plt.suptitle(f"Traffic Vectors for {proto}")
            #========================================================================
            #
            # the inbound traffic map...
            #
            #========================================================================
            worldmap.plot(color = "lightgrey", ax = ax[0])
            ax[0].set_xlim((-180.0, 180.0))
            ax[0].set_xlabel("Longitude", fontsize = pane_text)
            ax[0].set_ylim((-90.0, 90.0))
            ax[0].set_ylabel("Latitude", fontsize = pane_text)
            print(f"...inbound {proto} vectors")
            count = 0
            for i in range(0, npts):
                if (dire[i] == "inbound"):
                    count += 1
                    this_color = vector_color[i]
                    this_alpha = vector_alpha[i]
                    if (anomaly_scaling == "square"):
                        this_alpha = vector_alpha[i]**2
                    elif (anomaly_scaling == "sqrt"):
                        this_alpha = sqrt(vector_alpha[i])
                    elif (anomaly_scaling == "linear"):
                        this_alpha = vector_alpha[i]
                    ax[0].plot([slatlon[i][0], dlatlon[i][0]],
                               [slatlon[i][1], dlatlon[i][1]], 
                               color = this_color,
                               alpha = min((0.10 + this_alpha), 1), # this_alpha
                               lw = 4*(0.25 + 1.75 * this_alpha), # (0.25 + 5 * this_alpha),
                    )
            ax[0].set_title(f"{count} Inbound {proto} Vectors", fontsize = pane_title)
            #
            # create and render the legend...
            #
            legend_handles = []
            for i in range(0, len(graph_colors)):
                this_label = graph_labels[i]
                this_color = graph_colors[i]
                this_patch = mpatches.Patch(color = this_color, label = this_label)
                legend_handles.append(this_patch)
            ax[0].legend(loc = "lower left", handles = legend_handles)
            #========================================================================
            #
            # the outbound traffic map...
            #
            #========================================================================
            worldmap.plot(color = "lightgrey", ax = ax[1])
            ax[1].set_xlim((-180.0, 180.0))
            ax[1].set_xlabel("Longitude", fontsize = pane_text)
            ax[1].set_ylim((-90.0, 90.0))
            ax[1].set_ylabel("Latitude", fontsize = pane_text)
            print(f"...outbound {proto} vectors")
            count = 0
            for i in range(0, npts):
                if (dire[i] == "outbound"):
                    count += 1
                    this_color = vector_color[i]
                    this_alpha = vector_alpha[i]
                    if (anomaly_scaling == "square"):
                        this_alpha = vector_alpha[i]**2
                    elif (anomaly_scaling == "sqrt"):
                        this_alpha = sqrt(vector_alpha[i])
                    elif (anomaly_scaling == "linear"):
                        this_alpha = vector_alpha[i]
                    ax[1].plot([slatlon[i][0], dlatlon[i][0]],
                               [slatlon[i][1], dlatlon[i][1]], 
                               color = this_color,
                               alpha = min((0.10 + this_alpha), 1), # this_alpha
                               lw = 4*(0.25 + 1.75 * this_alpha), # (0.25 + 5 * this_alpha),
                    )
            ax[1].set_title(f"{count} Outbound {proto} Vectors", fontsize = pane_title)
            #
            # create and render the legend...
            #
            legend_handles = []
            for i in range(0, len(graph_colors)):
                this_label = graph_labels[i]
                this_color = graph_colors[i]
                this_patch = mpatches.Patch(color = this_color, label = this_label)
                legend_handles.append(this_patch)
            ax[1].legend(loc = "lower left", handles = legend_handles)
            #========================================================================
            #
            # save everything...
            #
            #========================================================================
            plt.savefig(os.path.join(places("images"), f"{proto}_traffic.png"))
            if (show_world):
                plt.show()
            else:
                plt.close()
    return None

def download_from_google_drive(shared_with_anyone_url:str, output_path:str) -> bool:
    """
    """
    cmdline = f"`which python | sed s/'\/python'/''/`/gdown --fuzzy {shared_with_anyone_url} --output {output_path}"
    print(f"Attempting to run '{cmdline}'")
    status = os.system(cmdline)
    if (status == 0):
        return True
    return False

In [3]:
## Fetch datafile if don't have locally...
dst = os.path.join(places("datasets"), "Dataset-Unicauca-Version2-87Atts.parquet.gzip")
if (not os.path.exists(dst)):
    src = "https://drive.google.com/file/d/1Z9M48CwJbbRyLEYUv-qwSx1kFJZPyF84/view?usp=sharing"
    download_from_google_drive(src, dst)
else:
    print(f"...found {dst} already present.")

...found /Volumes/blueprint/ncemc/battery/datasets/Dataset-Unicauca-Version2-87Atts.parquet.gzip already present.


In [4]:
## Tiny code fragment showing what's in the places("datasets") directory...
data_dir = places("datasets")
print(f"{data_dir} contains:")
for file_name in os.listdir(data_dir):
    file_path = os.path.join(data_dir, file_name)
    print(f"   {file_path}")

/Volumes/blueprint/ncemc/battery/datasets contains:
   /Volumes/blueprint/ncemc/battery/datasets/wx_stations_with_location.csv.gz
   /Volumes/blueprint/ncemc/battery/datasets/cloud_nodetypes.xls
   /Volumes/blueprint/ncemc/battery/datasets/cloud_nodetypes.csv
   /Volumes/blueprint/ncemc/battery/datasets/watcher_1727715107.parquet
   /Volumes/blueprint/ncemc/battery/datasets/tri_intel_monitor.parquet
   /Volumes/blueprint/ncemc/battery/datasets/the-verdict.txt
   /Volumes/blueprint/ncemc/battery/datasets/functional_test_dummy_data_metrics_history.parquet
   /Volumes/blueprint/ncemc/battery/datasets/old_ts300.parquet
   /Volumes/blueprint/ncemc/battery/datasets/covtype_train.parquet
   /Volumes/blueprint/ncemc/battery/datasets/antipattern_4_example_1.csv
   /Volumes/blueprint/ncemc/battery/datasets/clean_flt_ts3.csv.gz
   /Volumes/blueprint/ncemc/battery/datasets/regression_demo_holdout.csv
   /Volumes/blueprint/ncemc/battery/datasets/smoothing_test_2.csv
   /Volumes/blueprint/ncemc/batt

In [ ]:
populate_cache()

load addresses cache...
...loaded 0 addresses
len(lons) = 0
len(lats) = 0
load traffic data...
found /Volumes/blueprint/ncemc/battery/datasets/Dataset-Unicauca-Version2-87Atts.parquet.gzip, loading...
read dataframe with shape (3577296, 87) in 4.499 secs
...loaded 3577296 records


  0%|          | 1129/3577296 [05:00<322:55:35,  3.08it/s]

have read 100 new addresses, updating cache


  0%|          | 3130/3577296 [10:00<93:10:57, 10.65it/s] 

have read 100 new addresses, updating cache


...

  9%|▊         | 310186/3577296 [4:05:39<170:05:24,  5.34it/s]

have read 100 new addresses, updating cache


  9%|▊         | 312472/3577296 [4:10:40<108:24:31,  8.37it/s]

have read 100 new addresses, updating cache


  9%|▉         | 314169/3577296 [4:14:07<319:15:58,  2.84it/s]

In [ ]:
histograms_and_radar_plots(min_count = 1000, alpha_scaling = "square")

In [ ]:
create_traffic_maps_by_protocol(min_count = 1000, anomaly_scaling = "square")